In [1]:
from qiskit import QuantumCircuit
from qiskit.transpiler import CouplingMap
import random
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes.routing.sabre_swap_v0_20_      import SabreSwap 
from qiskit.transpiler.passes import ApplyLayout, FullAncillaAllocation, \
                                     EnlargeWithAncilla
from qiskit.transpiler.passes.layout.sabre_layout    import SabreLayout
from qiskit.qasm2 import dump 
from zou_circuits import create_parallel_circuit, apply_swaps_and_get_matching_circuit, scramble_qubits
random.seed(42)

In [2]:
num_qubits = 3
num_layers = 1
cm = CouplingMap.from_ring(num_qubits)


qc = create_parallel_circuit(num_qubits, num_layers, cm)

# Get the matching circuit after swaps are applied
matching_circuit = apply_swaps_and_get_matching_circuit(qc)

# Get transpiled circuit of the matching circuit
rp = SabreSwap(cm, seed=42)
lp = SabreLayout(cm, routing_pass=rp)

pm = PassManager([
    lp,
    FullAncillaAllocation(cm),
    EnlargeWithAncilla(),
    ApplyLayout(),
    rp
])

qc_transpiled = pm.run(matching_circuit)
qc_transpiled.draw()


# Print the original and modified circuits
print("Original Circuit:")
print(qc)
print("\nModified Circuit after applying SWAPs:")
print(matching_circuit.draw())
print("\nTranspiled Circuit of the modified circuit:")
print(qc_transpiled.draw())

Original Circuit:
             
q_0: ────────
     ┌───┐   
q_1: ┤ X ├─X─
     └─┬─┘ │ 
q_2: ──■───X─
             

Modified Circuit after applying SWAPs:
          
q_0: ─────
          
q_1: ──■──
     ┌─┴─┐
q_2: ┤ X ├
     └───┘

Transpiled Circuit of the modified circuit:
         ┌───┐
q_2 -> 0 ┤ X ├
         └─┬─┘
q_0 -> 1 ──┼──
           │  
q_1 -> 2 ──■──
              


In [3]:
print("\nOriginal Circuit Depth:", qc.depth())
print("Decomposed Original Circuit Depth:", qc.decompose().depth())

print("\nTranspiled Circuit Depth:", qc_transpiled.depth())
print("Decomposed Transpiled Circuit Depth:", qc_transpiled.decompose().depth())

print("\nModified Circuit Depth:", matching_circuit.depth())



Original Circuit Depth: 2
Decomposed Original Circuit Depth: 4

Transpiled Circuit Depth: 1
Decomposed Transpiled Circuit Depth: 1

Modified Circuit Depth: 1


In [4]:
def create_parallel_circuits_set(num_qubits, num_layers_list, coupling_map):
    # Create a set of quantum circuits with the given number of qubits and number of layers
    qc_list = [] # a list of tuples (original_circuit, matching_circuit)
    seed = 42
    for num_layers in num_layers_list:
        qc_orig = create_parallel_circuit(num_qubits, num_layers, coupling_map)
        file_answer = f"experiment_setup/circuits/zou_circuits/answers_ring_27_sample/parallel_circuit_{num_qubits}_{num_layers}.qasm"
        dump(qc_orig, file_answer)

        qc_match = apply_swaps_and_get_matching_circuit(qc_orig)
        file_problem  = f"experiment_setup/circuits/zou_circuits/problems_ring_27_sample/parallel_circuit_{num_qubits}_{num_layers}.qasm"
        dump(qc_match, file_problem)

        #file_scrambled = f"experiment_setup/circuits/zou_circuits/problems_ring_27_scrambled/parallel_circuit_{num_qubits}_{num_layers}.qasm"
        #qc_scrambled = scramble_qubits(qc_match, seed)
        #dump(qc_scrambled, file_scrambled)

        qc_list.append((qc_orig, qc_match))

        print("Done with num_layers = ", num_layers)
        seed += 1
    return qc_list

In [5]:
num_qubits = 27
num_layers_list = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
cm = CouplingMap.from_ring(num_qubits)


In [6]:
qc_list = create_parallel_circuits_set(num_qubits, num_layers_list, cm)

Done with num_layers =  10
Done with num_layers =  20
Done with num_layers =  30
Done with num_layers =  40
Done with num_layers =  50
Done with num_layers =  60
Done with num_layers =  70
Done with num_layers =  80
Done with num_layers =  90
Done with num_layers =  100
